In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
from six.moves import cPickle
import matplotlib.pyplot as plt

import tensorflow as tf
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency, metrics

import copy
import helper
import time

np.random.seed(247)
tf.set_random_seed(247)

# Load data

In [ ]:
data_path = '../data/Synthetic_dataset.h5'
results_path = '../results'

# load dataset
train, valid, test = helper.load_synthetic_dataset(data_path)

test_model = helper.load_synthetic_models(data_path, dataset='test')
    
# get data shapes
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = [None, train['targets'].shape[1]]

true_index = np.where(test['targets'][:,0] == 1)[0]
X = test['inputs'][true_index]
X_model = test_model[true_index]

# Train model

In [ ]:
 def perturb(x_nat, y, sess, nnmodel, feed_dict, grad_tensor, k=20):
    """Given a set of examples (x_nat, y), returns a set of adversarial
       examples within epsilon of x_nat in l_infinity norm."""
    epsilon = 0.2
    x = np.copy(x_nat)
    feed_dict[xx] = x
    feed_dict[yy] = y
    feed_dict[is_train] = False
    
    for i in range(k):
        feed_dict[xx] = x
        grad = sess.run(grad_tensor, feed_dict=feed_dict)

        x += 0.1 /(i+10) * np.sign(grad)

        x = np.clip(x, x_nat - epsilon, x_nat + epsilon) 
        x = np.clip(x, 0, 1) # ensure valid pixel range

    feed_dict[is_train] = True
    return x

def initialize_feed_dict(placeholders, feed_dict):

    train_feed = {}
    for key in feed_dict.keys():
        train_feed[placeholders[key]] = feed_dict[key]
    return train_feed


In [ ]:
all_models = ['LocalNet', 'DistNet'] #
dropout_status = [True, False]#,  True,   False,  False,  True,  False]
l2_status =      [True, False]#,  False,  True,   False,  False, True]
bn_status =      [True, False]#, False,  False,  True,   True,  True]

params_path = utils.make_directory(results_path, 'model_params')

batch_size = 50
verbose = 1 
num_epochs = 50
num_clean_epochs = 20
print_adv_test = True

adv_test = copy.deepcopy(test)

# get data shapes
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = [None, train['targets'].shape[1]]


# loop through models
for model_name in all_models:
    print('model: ' + model_name)

    for i in range(len(dropout_status)):
        tf.reset_default_graph()

        # compile neural trainer
        name = model_name
        if dropout_status[i]:
            name += '_do'
        if l2_status[i]:
            name += '_l2'
        if bn_status[i]:
            name += '_bn'
        name += '_adv'
        file_path = os.path.join(params_path, name)

        # load model parameters
        model_layers, optimization, _ = helper.load_model(model_name, 
                                                          input_shape,
                                                          dropout_status[i], 
                                                          l2_status[i], 
                                                          bn_status[i])


        # build neural network class
        nnmodel = nn.NeuralNet()
        nnmodel.build_layers(model_layers, optimization, supervised=True)

        grad_tensor = tf.gradients(nnmodel.mean_loss, nnmodel.placeholders['inputs'])[0]

        xx = nnmodel.placeholders['inputs']
        yy = nnmodel.placeholders['targets']
        is_train = nnmodel.placeholders['is_training']
        loss = nnmodel.mean_loss
        #   nnmodel.inspect_layers()
        performance = nn.MonitorPerformance('train', optimization['objective'], verbose)
        performance.set_start_time(start_time = time.time())

        train_calc = [nnmodel.train_step, loss, nnmodel.metric]
        train_feed = initialize_feed_dict(nnmodel.placeholders, nnmodel.feed_dict)

        # create neural trainer
        file_path = os.path.join(params_path, model_name)
        nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

        # initialize session
        sess = utils.initialize_session()

        # set data in dictionary
        #   data = {'train': train, 'valid': valid, 'test': test}
        x_train = train['inputs']
        y_train = train['targets']

        for epoch in range(num_epochs):
            print("Epoch: %d"%(epoch))
            index = np.random.permutation(len(x_train))

            for i in range((len(x_train) // batch_size)):
                # batch
                clean_batch_x = x_train[index[i*batch_size:(i+1)*batch_size]]
                clean_batch_y = y_train[index[i*batch_size:(i+1)*batch_size]]


                if epoch >= num_clean_epochs:
                    adv_batch = perturb(clean_batch_x, clean_batch_y,
                                sess, nnmodel, train_feed, grad_tensor)


                    train_feed[xx] = np.concatenate([clean_batch_x, adv_batch])
                    train_feed[yy] = np.concatenate([clean_batch_y, clean_batch_y])
                else:
                    train_feed[xx] = clean_batch_x
                    train_feed[yy] = clean_batch_y

                results = sess.run(train_calc, feed_dict=train_feed)
                performance.add_loss(results[1])
                #performance.progress_bar(i+1., (len(x_train) // batch_size), metric/(i+1))

            # print validation accuracy
            predictions = nntrainer.get_activations(sess, valid, 'output')
            print(metrics.accuracy(valid['targets'], predictions))

            if print_adv_test and epoch >= num_clean_epochs:
                adv_test['inputs'] = perturb(test['inputs'], test['targets'], sess, nnmodel, train_feed, grad_tensor)
        #       adv_test['inputs'] = test['inputs']
                predictions = nntrainer.get_activations(sess, adv_test, 'output')
                roc, roc_curves = metrics.roc(test['targets'], predictions)
                #print('Adversarial AUC')
                #print np.mean(roc)
                print('Adversarial Accuracy')
                print(metrics.accuracy(test['targets'], predictions))

            # save cross-validcation metrics
            loss, mean_vals, error_vals = nntrainer.test_model(sess, valid,
                                                                    name="valid",
                                                                    batch_size=batch_size,
                                                                    verbose=verbose)

        nnmodel.save_model_parameters(sess, os.path.join(params_path, model_name, name+'.ckpt'))

# Test model

In [ ]:
# Test model on held-out test set
loss, mean_vals, error_vals = nntrainer.test_model(sess, test,
                                                        name="test",
                                                        batch_size=batch_size,
                                                        verbose=verbose)

# Interpretability analysis

In [ ]:
methods = ['backprop', 'smoothgrad']
for method in methods:

    backprop_results ={}
    for model_name in all_models:

        for i in range(len(dropout_status)):

            # compile neural trainer
            name = model_name
            if dropout_status[i]:
                name += '_do'
            if l2_status[i]:
                name += '_l2'
            if bn_status[i]:
                name += '_bn'
            name += '_adv'
            file_path = os.path.join(params_path, name)

            # attribution parameters
            params = {'model_name': model_name, 
                      'input_shape': input_shape, 
                      'dropout_status': dropout_status[i],
                      'l2_status': l2_status[i],
                      'bn_status': bn_status[i],
                      'model_path': file_path+'.ckpt',
                     }

            # calculate attribution scores
            if method == 'smoothgrad':
                attribution_score = helper.smooth_backprop(X, params, layer='output', class_index=None, num_average=50)
            else:
                attribution_score = helper.backprop(X, params, layer='output', class_index=None, method=method)

            # calculate TPS and FPS 
            info = []
            for j, gs in enumerate(attribution_score):
                X_saliency = np.squeeze(gs).T
                tps, fps = helper.entropy_weighted_distance(X_saliency, X_model[j])
                info.append([tps, fps])
            info = np.array(info)

            backprop_results[name] = info

    # save results
    with open(os.path.join(results_path, method+'_adv.pickle'), 'wb') as f:
        cPickle.dump(backprop_results, f, protocol=cPickle.HIGHEST_PROTOCOL)